In [ ]:
import os
import re

import pandas as pd


In [313]:
file = open("text.txt", "r")
sentences = file.readlines()
sentences = [sentence.strip() for sentence in sentences]

df = pd.DataFrame(sentences, columns=['sentence'])


In [314]:
df.head()

,sentence
0,hindi movies for adults
1,are panda dogs real
2,asuedraw winning numbers
3,sentry replacement keys
4,rebuilding nicad battery packs


In [315]:
Q_WORD = ['what', 'where','when','why','who','how']
QQ_WORD = ['if','were','was','am','is','are','has','have','can','could','will','would','shall','should','do','does']
def detect_inquiry(sentences):
    result = []
    for sentence in sentences:
        flag = 0
        if(sentence[-1] == '?'):
            result.append(1)
            continue
        else:
            for word in Q_WORD:
                if(word in sentence.split(' ')):
                    result.append(1)
                    flag = 1
                    break
            
            for word in QQ_WORD:
                if(word == sentence.split(' ')[0]):
                    result.append(1)
                    flag = 1
                    break
            if(flag != 1):
                result.append(0)
            
    return result

In [316]:
result = detect_inquiry(sentences)
# print(result)

In [317]:
# res = pd.DataFrame(result, columns=['q'])

df['type'] = result

print(df.head())


                         sentence  type
0         hindi movies for adults     0
1             are panda dogs real     1
2        asuedraw winning numbers     0
3         sentry replacement keys     0
4  rebuilding nicad battery packs     0


In [319]:
df[df['type'] == 1].count()

sentence    1004
type        1004
dtype: int64

In [310]:
df['type'] = ['yes' if x == 1 else 'no' for x in df['type']]

In [ ]:
df[df['q'] == 'yes']

In [ ]:
df.to_csv('check.csv')

In [311]:
df

,sentence,type
0,hindi movies for adults,no
1,are panda dogs real,yes
2,asuedraw winning numbers,no
3,sentry replacement keys,no
4,rebuilding nicad battery packs,no
...,...,...
9988,sjogren syndrome pictures,no
9989,how do you renew a passport,yes
9990,medellin dim,no
9991,concursos funcab,no


import nltk
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()[:10000]


def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [261]:
import pandas as pd


In [262]:
ndf = pd.read_csv('file1.csv')
ndf

,statement,type
0,and confirm that address for me please,command
1,it's from Birmingham to em London Euston please,statement
2,the 8th of October,statement
3,i'd like to leave on the 7:33 train,statement
4,there's the 7:33 from Birmingham New Street,statement
...,...,...
210061,Come please,command
210062,Come on!,command
210063,Come on,command
210064,Don't eat so much.,command


In [263]:
ndf['type'] = [1 if t == 'question' else 0 for t in ndf['type']]
ndf[ndf['type'] == 1].count()

statement    130655
type         130655
dtype: int64

In [283]:
X_train, X_test, y_train, y_test = train_test_split(ndf['statement'], ndf['type'], test_size=0.2, random_state=0)

ndf = pd.concat([X_train, y_train], axis=1)
ndf.reset_index(drop=True, inplace=True)

In [284]:
ndf

,statement,type
0,certain armenians settled to sing non armenian...,0
1,however they too were eventually assimilated i...,0
2,the multidimensional pain inventory mpi is a q...,1
3,during what time was the cold war close to hap...,1
4,the west campus is located about one mile in w...,1
...,...,...
134436,on august windows was made available to downlo...,0
134437,when emperor haile selassie unilaterally disso...,1
134438,the compression works by reducing the accuracy...,1
134439,hunting is claimed to give resource managers a...,1


In [285]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


In [286]:
def sentence_transform(sentences):
    nsentences = []
    for sentence in sentences:
        sentence = re.sub("[^a-zA-Z]"," ",sentence)

        words = sentence.lower().split()

        nsentences.append(" ".join(words))
    return nsentences

ndf['statement'] = sentence_transform(ndf['statement'])

In [287]:
ndf

,statement,type
0,certain armenians settled to sing non armenian...,0
1,however they too were eventually assimilated i...,0
2,the multidimensional pain inventory mpi is a q...,1
3,during what time was the cold war close to hap...,1
4,the west campus is located about one mile in w...,1
...,...,...
134436,on august windows was made available to downlo...,0
134437,when emperor haile selassie unilaterally disso...,1
134438,the compression works by reducing the accuracy...,1
134439,hunting is claimed to give resource managers a...,1


In [288]:
vectorizer = CountVectorizer(analyzer='word',
        lowercase=True,
        tokenizer=None,
        stop_words=None,
        max_features=500,
        ngram_range=(1,2))

train_features = vectorizer.fit_transform(ndf['statement'])



In [289]:
train_features = train_features.toarray()

In [290]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(fit_prior=False,class_prior=None,alpha=0.5)


In [291]:
model.fit(train_features, ndf['type'])


MultinomialNB(alpha=0.5, fit_prior=False)

In [292]:
tdf = pd.concat([X_test, y_test], axis=1)
tdf.reset_index(drop=True, inplace=True)

tdf

,statement,type
0,barcelona restored some pride by beating real ...,0
1,current publications in the city include the a...,1
2,with the exception of lehman these companies r...,0
3,mortality has decreased over the last few deca...,0
4,there is also a crown inside the q and the who...,0
...,...,...
33606,what large species of antelope once made its h...,1
33607,the golden boot is awarded to the top premier ...,1
33608,what actor performed the role of archie bunker,1
33609,a typical pommel horse exercise involves both ...,1


In [293]:
test_transform = vectorizer.transform(tdf['statement'])

test_transform = test_transform.toarray()
predict = model.predict(test_transform)



In [294]:
from sklearn.metrics import classification_report

classification_report(predict , y_test)

'              precision    recall  f1-score   support\n\n           0       0.92      0.82      0.87     14330\n           1       0.88      0.95      0.91     19281\n\n    accuracy                           0.90     33611\n   macro avg       0.90      0.89      0.89     33611\nweighted avg       0.90      0.90      0.90     33611\n'

In [295]:
predict = predict.tolist()

In [296]:
tdf['ptype'] = predict

In [297]:
tdf

,statement,type,ptype
0,barcelona restored some pride by beating real ...,0,0
1,current publications in the city include the a...,1,0
2,with the exception of lehman these companies r...,0,0
3,mortality has decreased over the last few deca...,0,0
4,there is also a crown inside the q and the who...,0,0
...,...,...,...
33606,what large species of antelope once made its h...,1,1
33607,the golden boot is awarded to the top premier ...,1,0
33608,what actor performed the role of archie bunker,1,1
33609,a typical pommel horse exercise involves both ...,1,1


In [299]:
file = open("text.txt", "r")
sentences = file.readlines()
sentences = [sentence.strip() for sentence in sentences]

df = pd.DataFrame(sentences, columns=['sentence'])

In [300]:
test_transform = vectorizer.transform(df['sentence'])

test_transform = test_transform.toarray()
predict = model.predict(test_transform)



In [301]:
predict = model.predict(test_transform)

In [302]:
df['type'] = predict
df

,sentence,type
0,hindi movies for adults,0
1,are panda dogs real,1
2,asuedraw winning numbers,0
3,sentry replacement keys,0
4,rebuilding nicad battery packs,0
...,...,...
9988,sjogren syndrome pictures,0
9989,how do you renew a passport,1
9990,medellin dim,0
9991,concursos funcab,0


In [303]:
df[df['type'] == 1].count()

sentence    1523
type        1523
dtype: int64

In [312]:
import pickle 
pickle.dump(model, open('model.pkl','wb'))